In [3]:
import pandas as pd


df = pd.read_csv('data/name_gender_filtered.csv')
unique_chars = set()

for name in df['Name']:
    unique_chars.update(name)
    
sorted_chars = sorted(list(unique_chars))

In [4]:
sorted_chars = sorted(set(''.join(sorted_chars)))
stoi = {s:i for i,s in enumerate(sorted_chars)}
stoi['<S>'] = len(stoi)
stoi['<E>'] = len(stoi)
itos = {i:s for s,i in stoi.items()}
print(itos)

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: '<S>', 27: '<E>'}


In [13]:
import torch
import torch.nn.functional as F

def name_to_one_hot(name):
    # Add start and end tokens to the name
    tokenized_name = ['<S>'] + list(name) + ['<E>']
    int_tensor = torch.tensor([stoi[char] for char in tokenized_name])
    one_hot_encoded = F.one_hot(int_tensor, num_classes=len(stoi)).float()
    return one_hot_encoded


print(name_to_one_hot("leo"))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [6]:
from myRNN import *

n_letters = len(stoi)
n_hidden = 32
rnn_model = MyRNN(n_letters, n_hidden, 2)

In [12]:
prob = torch.tensor([0.7, 0.2, 0.1])

print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")
print(f"Sampled index: {torch.multinomial(prob, 1).item()}")

Sampled index: 0
Sampled index: 0
Sampled index: 2
Sampled index: 0
Sampled index: 0
Sampled index: 1
Sampled index: 0
Sampled index: 0
Sampled index: 0
Sampled index: 0


In [ ]:
@torch.no_grad()
def generate_name():
    rnn_model.eval()
    start_token_idx = torch.tensor(stoi['<S>'])
    one_hot_encoded = F.one_hot(start_token_idx, num_classes=len(stoi)).float()
    hidden = rnn_model.get_hidden()
    char_list = []
    for i in range(20):
        out_score, hidden = rnn_model(one_hot_encoded[None,:],hidden)
        score_probability = F.softmax(out_score[0], dim=-1)
        out_idx = torch.multinomial(score_probability, 1).item()
        if out_idx == stoi['<E>']:
            break
        char_list.append(itos[out_idx])
        one_hot_encoded = F.one_hot(torch.tensor(out_idx), num_classes=len(stoi)).float()
    print(''.join(char_list))


generate_name()

qjniyshieufurggrnre<S>


In [ ]:
from torch.optim import Adam

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_model.parameters(), lr=0.0001)

for epoch_idx in range(100):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)
    crnt_loss = 0.
    rnn_model.train()
    for index, row in shuffled_df.iterrows():

        name_one_hot = name_to_one_hot(row['Name'])
        hidden = rnn_model.get_hidden()
        rnn_model.zero_grad()

        losses = []
        for char_idx in range(len(name_one_hot)-1):
            input_tensor = name_one_hot[char_idx]
            target_char = name_one_hot[char_idx+1]
            target_class = torch.argmax(target_char, -1)
            out_score, hidden = rnn_model(input_tensor[None,:],hidden)
            losses.append(loss_fn(out_score[0], target_class))
        loss = sum(losses)
        loss.backward()
        optimizer.step()
        crnt_loss += loss.item()

    generate_name()
    average_loss = crnt_loss / len(df)

    print(f'Iter idx {epoch_idx}, Loss: {average_loss:.4f}')


In [ ]:
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()
generate_name()


ceairi
lewnor
valgwin
shala
ezmera
raffamau
kia
jenavieve
carlette
sheily
